In [1]:
# I am using Pytorch BTW.
import torch
from torch import nn
import torch.nn.functional as F

from tqdm import trange

%matplotlib inline
import matplotlib.pyplot as plt

# pip install madgrad
from madgrad import MADGRAD
from pcgrad import PCGrad

In [2]:
class NMF(nn.Module):
    def __init__(self, B, N, K):
        super(NMF, self).__init__()
        self.W = nn.Parameter(torch.rand(B, K, requires_grad=True, dtype=torch.float32))
        self.H = nn.Parameter(torch.rand(K, N, requires_grad=True, dtype=torch.float32))
    def forward(self):
        return torch.matmul(torch.tanh(self.W), self.H)

B = 100; N = 10; K = 2
Y = torch.rand(B, N)
nmf = NMF(B, N, K)

In [ ]:
optimizer = PCGrad(MADGRAD(nmf.parameters(), lr=1e-3, momentum=0.9))

l1loss = nn.L1Loss()

relu_func = nn.Threshold(0.0, 0.0)

epochs = 20000
for i in trange(epochs):
    optimizer.zero_grad()
    
    preds = nmf()
    l1 = l1loss(preds, Y)
    fro = torch.linalg.norm(preds-Y, 'fro')
    losses = [fro, l1]
    
    optimizer.pc_backward(losses)    
    optimizer.step()
    
    nmf.W.data = relu_func(nmf.W.data)
    nmf.H.data = relu_func(nmf.H.data)

 51%|█████     | 10108/20000 [00:08<00:08, 1197.95it/s]

In [ ]:
nmf.eval()
chk_loss = ((Y-nmf())**2).mean()
chk_loss.item()

#### Compare the performance with the Sklearn NMF function, giving close optimal solution

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
model = NMF(n_components=2, init='random', random_state=0)
W = model.fit_transform(Y.detach().numpy())
H = model.components_

In [ ]:
((W@H-Y.detach().numpy())**2).mean()